# Лабораторная работа 3. Нейронные сети

Результат лабораторной работы − отчет. Мы предпочитаем принимать отчеты в формате ноутбуков IPython (ipynb-файл). Постарайтесь сделать ваш отчет интересным рассказом, последовательно отвечающим на вопросы из заданий. Помимо ответов на вопросы, в отчете так же должен быть код, однако чем меньше кода, тем лучше всем: нам − меньше проверять, вам — проще найти ошибку или дополнить эксперимент. При проверке оценивается четкость ответов на вопросы, аккуратность отчета и кода.

### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 15 баллов. Сдавать задание после указанного срока сдачи нельзя. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов и понижают карму (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий в открытом источнике, необходимо прислать ссылку на этот источник (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, нам необходима ссылка на источник).

Обратите внимание, что мы не ставим оценку за просто написанный код, корректная работоспособность которого не подтверждена экспериментами. Работы, в которых удалены формулировки заданий, проверять сложнее, поэтому за решения без формулировок заданий мы будем снижать баллы в два раза.

### Правила сдачи
Выполненную работу следует отправить в систему Anytask. Более подробно о системе можно почитать на странице курса. Название отправляемого файла должно иметь следующий формат: Surname_Name_Group_NN.ipynb, где NN — номер лабораторной работы. Например, Kozlova_Anna_CS_03.ipynb.

## Алгоритм обратного распространения ошибки

Реализуйте небольшую библиотеку для обучения полносвязных нейронных сетей в рамках задачи классификации.

<img src="http://www.asimovinstitute.org/wp-content/uploads/2016/09/cnn.png" width=500></img>

Библиотека должна поддерживать описанный ниже набор базовых слоев и функций потерь. Обучение сети должно происходить методом стохастического градиентного спуска с подсчетом градиентов с помощью обратного распространения ошибки. На вход каждому слою передается вектор $x\in\mathbb{R}^n$.

### Поддерживаемые слои

 - Полносвязный (*Dense*) слой с заданием количества выходных нейронов $k$:
 
 $$\mathrm{Dense} \equiv \textbf{W}\textbf{x}+\textbf{b},$$
 
 где $\textbf{W}\in\mathbb{R}^{(k,n)}$ — матрица весов слоя, $\textbf{b}\in\mathbb{R}^k$ — вектор смещений (bias) слоя.
 
 
 - Слой логистической нелинейности (*Sigmoid*), который применяется поэлементно ко всем входам:
 
  $$\mathrm{Sigmoid} \equiv \frac{1}{1+e^{-\textbf{x}}}$$
  
 - Обобщение логистической функции для многомерного случая (*Softmax*). Позволяет моделировать распределение вероятностей над дискретным набором классов:
  
  $$\mathrm{Softmax} \equiv \frac{e^{-\textbf{x}_i}}{\sum\limits_{j = 1\dots k} e^{-\textbf{x}_j}}$$
  
 - [Dropout](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf) слой с заданием вероятности выключения нейрона ($d$). Dropout слой ведет себя по разному в фазай обучения и применения. 
  
  В фазе обучения независимо для каждого обучающего примера сэмплируется случайная бинарная маска $\textbf{m}$, которая будет определять какие нейроны окажутся выключенными:
  
    $$\mathrm{Dropout_{train}} \equiv \textbf{m}\odot\textbf{x}$$
    $$\textbf{m} \in \left\{0,1\right\}^{n}$$
    $$p\left(m_{i}=0\right)=d$$
   
  В фазе применения входные значения масштабируются, чтобы сохранить уровень активности, поступающий на следующий слой:
  
      $$\mathrm{Dropout_{test}}\equiv \left(1-d\right)\textbf{x}$$
      
#### Требования к реализации

Каждый из слоев должен быть написан в виде отдельного класса, с методами *fprop* (прямой просчет выходов сети) и *bprop* (обратный прогон сети с нахождением градиентов весовых коэффициентов):

    class Layer:
        def fprop(self, inputs, pass_type='train'):
            pass
            
        def bprop(self, outputs_deriv)
            pass
            
- В метод <i>bprop</i> должен передаваться градиент по выходам данного слоя. Для слоя softmax на вход (в переменной inputs) должны подаваться не только выходы предыдщего слоя, но и метки классов.

- При создании Dense слоя весовые коэффициенты должны инициализироваться из распределения $U(-0.01,0.01)$.

### Поддерживаемые функции потерь

 - Средний квадрат ошибки (*MSE*):
 
      $$\mathrm{L(y, \hat{y})} = \frac{1}{N} \sum\limits_{i=1}^N (\hat{y}_i - y_i)^2$$

 - Обобщение логистической регрессии на случай множества классов (*Cross-entropy*): 
     
     $$\mathrm{L(y, \hat{y})} = - \frac{1}{N} \sum\limits_{i=1}^N (y_i \log \hat{y}_i + (1-y_i) \log (1-\hat{y}_i))$$

### Общие требования к реализации библиотеки

Необходимо реализовать класс нейронной сети, который должен позволять конструировать сети из любой последовательности вышеназванных классов и обучаться на входной выборке. Класс нейронной сети должен хранить в себе упорядоченный список слоев.

    class NeuralNet:
        def add(self, layer):
            pass
    
        def fit(self, X_train, y_train, batch_size, lr, num_epochs):
            pass
            
        def predict(self, X):
            pass
            
 - Обучение должно происходить методом стохастического градиентного спуска с задаваемыми параметрами размера батча $K$ и шага обучения $\eta$:
$$\textbf{W} \leftarrow \textbf{W}-\eta \sum_{k=1}^{K}{\nabla_{\textbf{W}}L\left(\textbf{x}^k\right)}$$ 
(под батчем понимается группа примеров, которая обрабатывается одновременно без обновления весов; последний батч в эпохе может быть неполным)
 - В процессе обучения должен осуществляться подсчет как значения функции потерь, так и ошибки классификации. Эти ошибки должны выводиться один раз в эпоху. 
 - Методы *predict* и *fit* должны принимать на вход numpy матрицу, в строках которой содержатся обучающие примеры.
 - Все вычисления в описанных методах должны быть реализованы в матричной форме.

**Обратите внимание**, что в процессе написания библиотеки вам понадобится добавлять различные вспомогательные функции в исходный шаблон.

**(4 балла) Задание 1.** С помощью разработанных вами классов решите задачу классификации изображений [fashionMNIST](https://github.com/zalandoresearch/fashion-mnist). Ваша реализация должна работать эквивалентно по качеству [решению на основе keras](http://nbviewer.jupyter.org/urls/dl.dropbox.com/s/82656r55dx4pcoc/fMNIST_example.ipynb?dl=0) при той же самой архитектуре сети и параметрах обучения.

<img src="https://github.com/zalandoresearch/fashion-mnist/blob/master/doc/img/fashion-mnist-sprite.png?raw=true" width=500></img>

**(2 балла) Задание 2.** Используя возможности библитеки [keras](https://keras.io/), попробуйте достичь качества 88% на тесте, меняя следующие параметры модели:
- learning rate: постоянный (0.1 и 0.01) и переменный (метод [LearningRateScheduler](https://keras.io/callbacks/#learningratescheduler));
- тип активации на скрытом слое;
- количество скрытых слоев (от 1 до 6).

Для каждого эксперимента постройте графики качества и функции потерь. В экспериментах с количеством скрытых слоев сравните как возрастает время обучения при увеличении числа скрытых слоев.

## Классификация изображений на примере CIFAR-10

В этом задании вам предлагается попробовать различные архитектуры нейронных сетей в задаче классификации изображений. Для экспериментов будет использоваться база маленьких изображений [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html), состоящая из цветных изображений размером 32x32 пикселя и разделенная на 10 непересекающихся классов.

<img src="https://thkimorgblog.files.wordpress.com/2016/03/e18489e185b3e1848fe185b3e18485e185b5e186abe18489e185a3e186ba-2016-03-12-e1848be185a9e1848ce185a5e186ab-1-02-16.png" width=500>
</img>

В обучающей выборке содержится 50000 примеров по 5000 примеров на каждый класс. Тестовая выборка состоит из 10000 изображений.


Загрузите [файл](https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz) базы. В нем вы найдете обучающую выборку, разделенную на пять частей *data$\_$batch$\_$[1-5]*, и тестовую выборку *test_batch*. Каждую из частей можно загрузить с помощью следующей функции:

In [ ]:
import codecs
import cPickle

def unpickle(file):
    fo = codecs.open(file, 'rb')
    dict = cPickle.load(fo)
    fo.close()
    return dict

Каждая часть представляет из себя <i>dict</i> объект:
- <b>labels</b>: список меток изображений (цифры от 0 до 9)
- <b>data</b>: numpy массив изображений размерностью (10000, 3072) (то есть каждая строка представляет из себя изображение). Первые 1024 элемента массива соответствуют красному каналу изображения, вторые 1024 элемента $-$ зеленому, последние 1024 элемента $-$ синему. Сами изображения записаны построчно, то есть в рамках 1024 значений, соответствующих каждому из каналов, первые 32 значения $-$ это первая строчка изображения, вторые 32 значения $-$ вторая и т.д.

Загрузите датасет, объединив все обучающие батчи в одну матрицу.

**(1 балл) Задание 3.** С помощью библиотеки [keras](https://keras.io/) обучите небольшую полносвязную сеть на базе CIFAR-10. Сеть должна состоять из двух слоев по 512 нейронов, с функцией активации [ReLU](https://en.wikipedia.org/wiki/Rectifier_(neural_networks). Для обучения сети используйте стохастический градиентный спуск с [моментумом Нестерова](https://keras.io/optimizers/#sgd). Попытайтесь подобрать начальный <i>learning rate</i>, темп его снижения <i>decay</i>, а также *коэффициенты l2 регуляризаторов* на весовые коэффициенты сети так, чтобы получить наилучшее качество классификации на тестовой выборке (значение функции потерь cross-entropy и топ-1 ошибку классификации). Диапазоны параметров, с которыми рекомендуется провести эксперименты:
- <i>learning rate</i>: от 0.005 до 0.1
- *decay*: от 1e-9 до 1e-5
- *l2 регуляризация*: от 1e-6 до 1e-4

Перед обучением необходимо нормализовать обучающую выборку так, чтобы все входные значения были в интервале [0;1].

**(1 балл) Задание 4.** Поэкспериментируйте с размером сети: попробуйте увеличить количество нейронов в каждом слое и общую глубину сети.  Наблюдается ли сильное повышение качества классификации на тестовой выборке?

Теперь давайте попробуем обучить сверточную сеть небольшого размера. Для этого необходимо вначале изменить формат наших данных, так как сверточные сети работают не с векторами примеров, а с многомерными матрицами примеров. Для этого каждый элемент выборки нужно преобразовать из вектора длины 3072 в матрицу с размерностями (3, 32, 32), где каждый из элементов первой размерности отвечает за свой канал изображения.

**(2 балла) Задание 5.** Первая сверточная сеть будет состоять из трех сверточных блоков $-$ последовательностей свертки ([Convolution2D](https://keras.io/layers/convolutional/#conv2d)), нелинейности (ReLU) и пулинга ([MaxPooling2D](https://keras.io/layers/pooling/#maxpooling2d)). При построении сверточной нейронной сети обычно необходимо определить значения нескольких параметров:
- количества ядер свертки (различных карт признаков) в каждом из слоев
- размеры ядер
- тип и размер области слоя пулинга

**Карта признаков** (feature map) $-$ это результат применения одного ядра свертки ко входам в слой, которые в свою очередь представляют собой множество карт признаков предыдущего слоя.

Для текущего задания возьмите следующие значения:
- *размеры фильтров* во всех сверточных слоях будут равными 5x5 пикселей;
- *размер области max-пулинга* 3x3 пикселя со смещением в 2 пикселя (области пулинга будут перекрываться);
- *количество ядер* в первой сверточном слое возьмем равным 48, во втором — 64, а в третьем — 128.

**На практике** обычно используют пулинг размером 2x2 без перекрытия областей. Применение такого пулинга приводит к уменьшению в два раза размерности каждой из входных карт признаков. В следующем после пулинга сверточном слое увеличивают количество ядер в два раза. Однако в данной работе необходимо сделать достаточно небольшую и неглубокую сеть, которую можно обучить за разумное время на обычном ноутбуке, поэтому в данном задании мы отступаем от стандартных практик.

После сверточных блоков необходимо добавить в сеть два полносвязных слоя размерности 512 с нелинейностями.
Перед каждым сверточным слоем в сети необходимо добавить слои дополнения нулями входных карт признаков, для того чтобы после операции свертки размеры карт не изменялись, используя слой [ZeroPadding2D](https://keras.io/layers/convolutional/#zeropadding2d). Подберите значения паддинга таким образом, чтобы после свертки 5x5 получались карты признаков такой же размерности, как и до свертки.

Итоговую архитектуру сети можно вывести после применения метода <i>compile</i> с помощью:
    
    print model.summary()
    
Попробуйте подобрать начальный learning rate так, чтобы избежать расхождения обучения в самом начале.

**Обратите внимание:**
- обучение подобной сети на ноутбуке в течение 15 эпох может занять около 3-х часов. Для того, чтобы убедиться в корректности составленной архитектуры можно попробовать запустить обучение на подмножестве обучающей выборки и убедиться в уменьшении ошибки классификации;
- для сверточных сетей критерий останова вводить не стоит, так как обучение их до сходимости весьма затратно по времени.

Большие нейронные сети склонны к переобучению. Для того, чтобы бороться с переобучением, в области распознавания изображений активно применяют техники расширения обучающей выборки. Для этого к обучающим изображениям применяют различные преобразования, которые не меняют класс объекта. В рамках данного задания мы возьмем сверточную сеть, которую обучали ранее, и будем применять ко входным изображениям случайные сдвиги по обеим координатам и горизонтальные перевороты изображения. Для этого можно воспользоваться специальным классом [ImageDataGenerator](https://keras.io/preprocessing/image/#imagedatagenerator). 

Задайте параметры <i>ImageDataGenerator</i> таким образом, чтобы он осуществлял случайные горизонтальные и вертикальные сдвиги изображения в пределах трех пикселей и случайные горизонтальные перевороты. Также не забудьте правильно выставить порядок размерностей во входных данных в параметре <i>dim_ordering</i> при создании экземпляра класса ImageDataGenerator.

**(1 балл) Задание 6.** Обучите сверточную сеть той же архитектуры, которая использовалась раннее. Как расширение выборки отразилось на качестве классификации на тестовой выборке?

## Предсказание тональности отзывов на фильмы

В данном задании вам предлагается попробовать различные архитектуры нейронных сетей и форматы представления входных данных в задаче текстовой классификации на примере [базы отзывов IMDB](http://ai.stanford.edu/~amaas/data/sentiment/).

<img src="http://medialeaks.ru/wp-content/uploads/2015/10/IMDb.jpg" width=500>
</img>

Данная база содержит 25000 отзывов в обучающем множестве и 25000 отзывов в тестовом. Все отзывы разбиты на два класса: положительные и отрицательные. Данные разбиты между обучающим и тестовым множествам по фильмам, то есть в тестовом множестве нет отзывов на те фильмы, которые есть в обучении.

В файле с базой есть две разные папки для обучающей и тестовой выборке. В каждой из папок верхнего уровне есть папки отрицательных и положительных отзывов. Каждый отзыв лежит в отдельном текстовом файле.

Загрузите отзывы из обучающей и тестовой выборок. Напишите схему токенизации текста:
- текст нужно перевести в нижний регистр
- выделить слова в тексте. 

Словами/токенами мы считаем последовательности букв и цифр, разделенные знаками препинания и пробелами. Здесь и далее для токенизации текста вы можете использовать функцию [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html).

Выделите множество токенов в обучающих текстах и составьте словарь из 10000 самых частотных слов.

В качестве представления для входных текстов мы будем использовать мешок слов. Каждый отзыв будет представлен вектором с длиной равной количеству токенов в словаре. На позиции в векторе, которая соответствует определенному токену в словаре, будет записано количество вхождений этого токена в обучающий сэмпл. Так как keras не поддерживает разреженные входы, то необходимо будет использовать обычные вектора. Обратите внимание, что в случае использования функции *CountVectorizer*, необходимо дополнительное преобразование из разряженного представления. Напишите функцию, которая по входному тексту и словарю возвращает его представление в виде мешка слов.

**(1 балл) Задание 7.** С помощью функций [train_on_batch](https://keras.io/models/sequential/#train_on_batch) и [test_on_batch](https://keras.io/models/sequential/#test_on_batch) класса модели в keras напишите процедуру обучения/тестирования нейронной сети в побатчевом режиме на выборке сэмплов. 

Функция должна принимать на вход выборку текстов и соответствующих им классификационных меток, а также размер батча, используемого при обучении/тестировании. Подача  батчей в сеть должна происходить в случайном порядке. Кодирование текстов в сэмплах должно осуществляться непосредственно в момент формирования батча. Это позволяет значительно снизить необходимый объем памяти для обработки выборки, так как не обязывает нас хранить все сэмплы в закодированном виде.

Также данная процедура должна осуществлять подсчет функци потерь и классификационной ошибки на обучающей выборке путем усреднения ошибок на отдельных батчах. 

**(1 балл) Задание 8.** С помощью разработанного метода обучите полносвязную сеть для решения задачи предсказания тональности. Попробуйте использовать различные архитектуры (с разным количеством слоев, например, 2-4, и нейронов, например, от 100 до 500), а также разные методы регуляризации (l2 регуляризация на веса и Dropout, в том числе после самого первого слоя). Попробуйте провести эксперименты не только с обычным градиентным спуском, но и с более сложными алгоритмами с адаптивными learning rate, например, [RMSPROP](https://keras.io/optimizers/#rmsprop). 

Для быстрой оценки качества подобранной комбинации иногда достаточно посмотреть на динамику ошибки в течение первых нескольких эпох обучения. При этом однако, необходимо помнить, что более агрессивная регуляризация обычно снижает скорость сходимости, но приводит к более низкой ошибке на тестовой выборке.

Сделайте небольшую таблицу с результатами вашего исследования. Какого наилучшего качества классификации вам удалось добиться на тестовой выборке и с какой комбинацией параметров?

Так как выборка IMDB является не очень большой по размеру, то используя только ее достаточно сложно выучить хорошие распределенные представления для слов, которые формируются в первом полносвязном слое сети.

Мы можем инициализировать этот первый слой сети из каких-либо предобученных векторных представлений слов. В рамках данного задания мы будем использовать 100000 наиболее частотных слов из модели word2vec, обученной на большом корпусе новостей. Скачать вектора можно по [ссылке](https://www.dropbox.com/s/wefgvcqqym0brh7/mikolov_w2v_100k.tsv?dl=0). В этом файле в первой строчке записано количество слов в словаре и размерность представления, а дальше на на каждой строчке записано слово, а за ним идет 300-мерное представление этого слова. 

Пересеките выделенный вами словарь со словарем word2vec и составьте матрицу весов для инициализации первого полносвязного слоя сети. Более 80% слов из вашего словаря будут присутствовать и в word2vec, а вектора тех слов, которые не нашлись, проинициализируйте из распределения $U\left(-0.05, 0.05\right)$

**(0.5 балла) Задание 9.** Обучите нейронную сеть с подобранными вами ранее наилучшими параметрами, при этом инициализировав первый Dense слой получившейся матрицей. Так как предоставленный word2vec имеет размерность 300, то и этот первый слой также будет иметь 300 нейронов. Удалось ли вам получить более высокое качество классификации?

Небольшое количество слов с словаре не позволяет учитывать все возможные словоформы различных слов, а также частые опечатки, которые встречаются в реальных текстах. Особенно это становится заметно на языках с богатой морфологией. Из-за этого редкие слова не попадают в мешок слов, что потенциально может снижать качество работы классификатора.

Для борьбы с этим явлением на практике в дополнение к словам часто используются буквенные триграммы в качестве токенов. Например, текст "Машинное обучение - рулит!" после приведения к нижнему регистру будет разбит на множество триграмм ["маш", "аши", "шин", "инн", "ное", "ое ", "е о", " об", "обу", "буч", "уче", "чен", "ени", "ние", "ие ", "е -", " - ", "- р", " ру", "рул", "ули", "лит", "ит!"]. Также буквенные триграммы позволяют обучать совместное представления для однокоренных слов. Обычно при составлении буквенных триграмм мы не игнорируем знаки пунктуации и пробелы в тексте.

Выделите из обучающих текстов 2000 наиболее частотных буквенных триграмм (после приведения текстов в нижний регистр). Составьте смешанный словарь токенов, в котором будет 8000 наиболее частотных слов и 2000 триграмм. 

**(0.5 балла) Задание 10.** Модифицируйте методы кодирования текста и метод обучения нейронной сети так, чтобы они работали для смешанного словаря. Попробуйте обучить несколько архитектур полносвязных нейронных сетей, используя новое представления текста.

**(1 балл) Задание 11.**Постройте сводный график, сравнивающий различные представления текста и различные архитектуры сетей. Какие выводы можно сделать из проведенного вами исследования?